In [1]:
# セル 1: 依存関係のインストール (バージョンを調整)
!pip install flask pyngrok -U -qq

# PyTorch と torchvision のバージョンを指定 (Colab GPU環境向け)
# CUDA 11.8 環境 (T4 GPUなど) を想定
!pip install torch==2.1.0 torchvision==0.16.0 torchaudio==2.1.0 --index-url https://download.pytorch.org/whl/cu118 -U -qq
# Colabの環境によっては CUDA 12.1 (cu121) が適切な場合もあります。!nvcc --version で確認できます。

# Transformers と関連ライブラリ (sentence-transformers と互換性のあるバージョンを指定)
# sentence-transformers 3.4.1 は transformers>=4.41.0 を要求するため、4.41.2 を試す
!pip install transformers==4.41.2 accelerate sentencepiece google-colab -U -qq # <- ここを変更

In [4]:
import os
import logging
import torch
from flask import Flask, request, jsonify, render_template_string # render_template_stringに変更
from pyngrok import ngrok, conf, exception as pyngrok_exception
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig # 量子化を使う場合
from google.colab import drive, userdata

# --- 設定項目 ---
MODEL_NAME = "rinna/llama-3-youko-8b"
# MODEL_NAME = "stabilityai/japanese-stablelm-2-1_6b" # 必要に応じて軽量なモデルに変更
MODEL_DIR = "/content/drive/MyDrive/llama_model_cache" # Google Drive内のキャッシュディレクトリ名（任意）
NGROK_SECRET_NAME = "NGROK_AUTHTOKEN" # ステップ1で設定したColabシークレット名
FLASK_PORT = 5000 # Flaskが使用するポート
USE_GOOGLE_DRIVE_CACHE = True # Google Driveをキャッシュとして使うか
# --- ここまで ---

# セル 2: インポートと基本設定 の logging 設定部分を修正
logging.basicConfig(
    level=logging.INFO, # INFOレベル以上を出力
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
    force=True # Colab環境で設定を確実に上書き
)
# pyngrokのロガーもINFOに設定（何が起きているか少し詳しく見るため）
logging.getLogger("pyngrok").setLevel(logging.INFO)

logger = logging.getLogger(__name__) # このモジュール用のロガーを取得

# グローバル変数 (初期化)
tokenizer = None
model = None
ngrok_tunnel = None # ngrokトンネルオブジェクト

In [5]:
if USE_GOOGLE_DRIVE_CACHE:
    try:
        drive.mount('/content/drive', force_remount=True)
        logger.info("Google Drive mounted successfully at /content/drive.")
        # キャッシュディレクトリが存在しない場合は作成
        cache_path = os.path.join(MODEL_DIR)
        os.makedirs(cache_path, exist_ok=True)
        logger.info(f"Model cache directory set to: {cache_path}")
    except Exception as e:
        logger.error(f"Failed to mount Google Drive: {e}", exc_info=True)
        logger.warning("Proceeding without Google Drive model caching.")
        MODEL_DIR = None # Driveが使えない場合はキャッシュ無効化
else:
    logger.info("Google Drive caching is disabled.")
    MODEL_DIR = None

2025-05-03 03:50:11 - __main__ - INFO - Google Drive mounted successfully at /content/drive.
2025-05-03 03:50:11 - __main__ - INFO - Model cache directory set to: /content/drive/MyDrive/llama_model_cache


Mounted at /content/drive


In [6]:
try:
    # Colabシークレットからトークンを取得
    ngrok_token = userdata.get(NGROK_SECRET_NAME)
    if not ngrok_token:
         raise ValueError(f"'{NGROK_SECRET_NAME}' value is empty in Colab secrets.")

    # pyngrokのデフォルト設定に認証トークンをセット
    conf.get_default().auth_token = ngrok_token
    logger.info("Ngrok authentication token set successfully from Colab secrets.")

except userdata.SecretNotFoundError:
    logger.error(f"Fatal: Ngrok authtoken secret '{NGROK_SECRET_NAME}' not found in Colab secrets.")
    logger.error("Please follow Step 1 to add your ngrok authtoken as a secret.")
    # トークンがないとngrokは使えないのでエラーにする
    raise SystemExit("Ngrok authtoken is required to proceed.")
except ValueError as e:
    logger.error(f"Fatal: {e}")
    raise SystemExit("Ngrok authtoken configuration error.")
except Exception as e:
    logger.error(f"Fatal: An unexpected error occurred while setting ngrok auth token: {e}", exc_info=True)
    raise SystemExit("Ngrok authtoken configuration error.")

2025-05-03 03:50:17 - __main__ - INFO - Ngrok authentication token set successfully from Colab secrets.


In [7]:
def prepare_model_and_tokenizer(model_name, cache_dir):
    """モデルとトークナイザーをロードまたはダウンロードする"""
    global tokenizer, model
    logger.info(f"Preparing model and tokenizer for: {model_name}")

    # 量子化設定 (メモリ削減のため、必要に応じて有効化)
    # bnb_config = BitsAndBytesConfig(
    #     load_in_4bit=True,
    #     bnb_4bit_use_double_quant=True,
    #     bnb_4bit_quant_type="nf4",
    #     bnb_4bit_compute_dtype=torch.bfloat16
    # )

    model_load_path = model_name # デフォルトはHuggingFace Hubから
    load_from_cache = False

    # Google Driveキャッシュが有効で、キャッシュが存在するかチェック
    if cache_dir and os.path.exists(os.path.join(cache_dir, model_name.replace('/', '_'), "config.json")):
        model_load_path = os.path.join(cache_dir, model_name.replace('/', '_'))
        logger.info(f"Found model cache in Google Drive: {model_load_path}")
        load_from_cache = True
    elif cache_dir:
        logger.info(f"Model cache not found in {cache_dir}. Will download and potentially save.")
        # 保存先パスだけ設定
        model_save_path = os.path.join(cache_dir, model_name.replace('/', '_'))
    else:
        logger.info("Google Drive cache is disabled or unavailable. Loading directly from HuggingFace Hub.")
        model_save_path = None # 保存しない

    try:
        logger.info(f"Loading tokenizer from: {model_load_path}")
        tokenizer = AutoTokenizer.from_pretrained(model_load_path)

        logger.info(f"Loading model from: {model_load_path}")
        model = AutoModelForCausalLM.from_pretrained(
            model_load_path,
            torch_dtype=torch.bfloat16, # BF16を使用 (対応GPUが必要)
            device_map="auto",          # 自動でデバイス (GPU/CPU) に割り当て
            # quantization_config=bnb_config, # 量子化を使う場合コメント解除
            low_cpu_mem_usage=True,     # CPUメモリ使用量を抑える試み
        )
        logger.info("Model and tokenizer loaded successfully.")

        # キャッシュからロードしなかった場合で、保存先が指定されていれば保存
        if not load_from_cache and model_save_path:
             try:
                 logger.info(f"Saving model and tokenizer to cache: {model_save_path}")
                 os.makedirs(model_save_path, exist_ok=True)
                 tokenizer.save_pretrained(model_save_path)
                 model.save_pretrained(model_save_path)
                 logger.info("Successfully saved model and tokenizer to cache.")
             except Exception as save_e:
                 logger.error(f"Failed to save model/tokenizer to cache directory {model_save_path}: {save_e}", exc_info=True)

    except Exception as e:
        logger.error(f"Fatal error during model/tokenizer preparation: {e}", exc_info=True)
        if "out of memory" in str(e).lower():
            logger.error(">>> Hint: You might be running out of GPU memory. Try a smaller model or enable quantization.")
        raise SystemExit("Failed to prepare model and tokenizer.")

# 実行
prepare_model_and_tokenizer(MODEL_NAME, MODEL_DIR)

2025-05-03 03:50:27 - __main__ - INFO - Preparing model and tokenizer for: rinna/llama-3-youko-8b
2025-05-03 03:50:27 - __main__ - INFO - Found model cache in Google Drive: /content/drive/MyDrive/llama_model_cache/rinna_llama-3-youko-8b
2025-05-03 03:50:27 - __main__ - INFO - Loading tokenizer from: /content/drive/MyDrive/llama_model_cache/rinna_llama-3-youko-8b
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
2025-05-03 03:50:28 - __main__ - INFO - Loading model from: /content/drive/MyDrive/llama_model_cache/rinna_llama-3-youko-8b
2025-05-03 03:50:28 - accelerate.utils.modeling - INFO - We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

2025-05-03 03:50:40 - __main__ - INFO - Model and tokenizer loaded successfully.


In [15]:
app = Flask(__name__)

# --- HTMLテンプレート ---
# render_template_stringを使うため、HTMLをPython文字列として定義
chat_html_template = """
<!DOCTYPE html>
<html lang="ja">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Chatbot Interface</title>
    <style>
        body { font-family: sans-serif; margin: 0; padding: 0; background-color: #f0f2f5; display: flex; justify-content: center; align-items: center; min-height: 100vh; }
        .chat-container { background-color: #fff; box-shadow: 0 4px 12px rgba(0,0,0,0.1); border-radius: 8px; width: 90%; max-width: 700px; height: 85vh; display: flex; flex-direction: column; overflow: hidden; }
        h1 { text-align: center; color: #1c1e21; margin: 20px 0; font-size: 1.6em; font-weight: 600; }
        #chat-box { flex-grow: 1; padding: 20px; overflow-y: auto; border-top: 1px solid #dddfe2; border-bottom: 1px solid #dddfe2; background-color: #fff; }
        .message { margin-bottom: 15px; display: flex; flex-direction: column; max-width: 85%; word-wrap: break-word; }
        .message span { padding: 10px 15px; border-radius: 18px; line-height: 1.4; font-size: 0.95em; }
        .user { align-self: flex-end; }
        .user span { background-color: #0084ff; color: white; border-bottom-right-radius: 5px; }
        .bot { align-self: flex-start; }
        .bot span { background-color: #e4e6eb; color: #050505; border-bottom-left-radius: 5px; }
        .thinking span { background-color: #f0f0f0; color: #65676b; font-style: italic; }
        .error span { background-color: #fdeded; color: #d32f2f; border: 1px solid #f5c6cb; }
        .input-area { display: flex; padding: 15px; border-top: 1px solid #dddfe2; background-color: #f0f2f5; }
        #user-input { flex-grow: 1; padding: 12px 18px; border: 1px solid #ccd0d5; border-radius: 20px; margin-right: 10px; font-size: 1em; outline: none; transition: border-color 0.2s ease; }
        #user-input:focus { border-color: #0084ff; }
        #send-button { padding: 12px 20px; background-color: #0084ff; color: white; border: none; border-radius: 20px; cursor: pointer; font-size: 1em; transition: background-color 0.2s ease; font-weight: 600; }
        #send-button:hover:not(:disabled) { background-color: #0073e0; }
        #send-button:disabled { background-color: #bcc0c4; cursor: not-allowed; }
    </style>
</head>
<body>
    <div class="chat-container">
        <h1>AI Chatbot</h1>
        <div id="chat-box">
             <div class="message bot"><span>こんにちは！メッセージを入力してください。</span></div>
        </div>
        <div class="input-area">
            <input type="text" id="user-input" placeholder="ここにメッセージを入力..." autocomplete="off">
            <button id="send-button" onclick="sendMessage()">送信</button>
        </div>
    </div>

    <script>
        const chatBox = document.getElementById('chat-box');
        const userInput = document.getElementById('user-input');
        const sendButton = document.getElementById('send-button');

        // メッセージをチャットボックスに追加する関数
        function addMessage(text, type) {
            const messageDiv = document.createElement('div');
            messageDiv.className = `message ${type}`;
            // テキスト内の改行を<br>に変換して表示
            const formattedText = text.replace(/\\n/g, '<br>');
            messageDiv.innerHTML = `<span>${formattedText}</span>`;
            chatBox.appendChild(messageDiv);
            // 遅延させてからスクロールすることで、レンダリング後に確実に最下部に移動
            setTimeout(() => { chatBox.scrollTop = chatBox.scrollHeight; }, 50);
            return messageDiv;
        }

        // メッセージ送信処理
        async function sendMessage() {
            const messageText = userInput.value.trim();
            if (!messageText) return; // 空のメッセージは無視

            addMessage(messageText, 'user'); // ユーザーメッセージを表示
            userInput.value = ''; // 入力欄をクリア
            userInput.disabled = true; // 入力と送信を無効化
            sendButton.disabled = true;

            const thinkingMessage = addMessage("考え中...", 'thinking bot'); // 思考中メッセージ

            try {
                // /chatエンドポイントにPOSTリクエスト
                const response = await fetch("/chat", {
                    method: "POST",
                    headers: { "Content-Type": "application/json" },
                    body: JSON.stringify({ message: messageText })
                });

                chatBox.removeChild(thinkingMessage); // 思考中メッセージを削除

                const data = await response.json(); // レスポンスをJSONとして解析

                if (!response.ok) {
                    // サーバーからのエラーレスポンス
                    addMessage(`エラー: ${data.error || response.statusText || '不明なサーバーエラー'}`, 'error bot');
                    console.error("Server error response:", data);
                } else {
                    // 正常な応答
                    addMessage(data.response || "応答がありませんでした。", 'bot');
                }
            } catch (error) {
                // ネットワークエラーなど
                chatBox.removeChild(thinkingMessage); // 思考中メッセージを削除
                addMessage("通信エラーが発生しました。接続を確認してください。", 'error bot');
                console.error("Fetch API error:", error);
            } finally {
                // 応答後に入力と送信を再度有効化
                userInput.disabled = false;
                sendButton.disabled = false;
                userInput.focus(); // 入力欄にフォーカス
                // 最終スクロール
                setTimeout(() => { chatBox.scrollTop = chatBox.scrollHeight; }, 50);
            }
        }

        // Enterキーで送信
        userInput.addEventListener('keypress', function(event) {
            if (event.key === 'Enter' && !sendButton.disabled) {
                sendMessage();
            }
        });

        // ページ読み込み時に入力欄にフォーカス
        userInput.focus();
    </script>
</body>
</html>
"""
# --- ここまでHTML ---

@app.route("/")
def home():
    """チャットUIのHTMLを表示するルート"""
    # render_template_stringを使用してPython文字列内のHTMLをレンダリング
    return render_template_string(chat_html_template)

@app.route("/chat", methods=["POST"])
def chat():
    """チャットリクエストを受け取り、モデルの応答を返すAPIエンドポイント"""
    global tokenizer, model
    # モデルがロードされているか確認
    if not tokenizer or not model:
         logger.error("Chat request received but model/tokenizer is not ready.")
         return jsonify({"error": "モデルが準備できていません。管理者に連絡してください。"}), 503 # Service Unavailable

    try:
        # リクエストボディからメッセージを取得
        user_input = request.json.get("message", "").strip()
        logger.info(f"Received chat request with input: '{user_input}'")
        if not user_input:
            logger.warning("Received empty input message.")
            return jsonify({"error": "メッセージを入力してください。"}), 400 # Bad Request

        # Rinna Llama 3 Youko 用のプロンプト形式
        # https://huggingface.co/rinna/llama-3-youko-8b#%E6%8E%A8%E8%AB%96%E6%96%B9%E6%B3%95
        prompt = (
            "<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\n"
            f"{user_input}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"
        )

        # 入力をトークン化してモデルと同じデバイスに送る
        inputs = tokenizer(prompt, return_tensors="pt", add_special_tokens=False).to(model.device)
        # add_special_tokens=False にしてプロンプトテンプレートの特殊トークンと重複しないように

        # モデル応答生成
        logger.info("Generating model response...")
        with torch.no_grad(): # 勾配計算を無効化
            outputs = model.generate(
                **inputs,
                max_new_tokens=50,  # 生成トークン数を少し増やす
                temperature=0.7,     # 温度 (低いほど決定的、高いほど多様)
                top_p=0.9,           # Top-pサンプリング
                do_sample=True,      # サンプリングを有効化
                pad_token_id=tokenizer.eos_token_id, # パディングID
                # repetition_penalty=1.1, # ★★★ 繰り返し抑制ペナルティを追加 ★★★
                # 停止トリガーとなるトークンID
                eos_token_id=[tokenizer.eos_token_id, tokenizer.convert_tokens_to_ids("<|eot_id|>")]
            )
        logger.info("Model response generated.")

        # 生成された応答全体をデコード (特殊トークンも含めて)
        full_response = tokenizer.decode(outputs[0], skip_special_tokens=False)
        logger.debug(f"Raw model output: {full_response}")

        # --- 応答の後処理 ---
        # アシスタント応答の開始位置を探す (プロンプトテンプレートに対応)
        assistant_marker = "<|start_header_id|>assistant<|end_header_id|>\n\n"
        # 応答は入力の後に追加されるので、rfindで最後に出現するものを探す
        response_start_index = full_response.rfind(assistant_marker)

        if response_start_index != -1:
            # マーカーが見つかった場合、マーカー以降を抽出
            response_text = full_response[response_start_index + len(assistant_marker):]
        else:
            # マーカーが見つからない場合 (フォールバック)
            logger.warning("Assistant marker not found in response. Attempting to remove prompt manually.")
            # 入力プロンプトで始まっているか確認して除去
            if full_response.startswith(prompt):
                 response_text = full_response[len(prompt):]
            else:
                 response_text = full_response # そのまま返す (望ましくない可能性)

        # 末尾の不要な特殊トークン (<|eot_id|> や <|end_of_text|>) を除去
        response_text = response_text.replace("<|eot_id|>", "").replace("<|end_of_text|>", "").strip()
        # --- 後処理ここまで ---

        logger.info(f"Processed response: '{response_text}'")
        # 応答をJSONで返す
        return jsonify({"response": response_text})

    except Exception as e:
        logger.error(f"Error during chat processing: {e}", exc_info=True)
        # ユーザーには一般的なエラーメッセージを返す
        return jsonify({"error": "応答の生成中に内部エラーが発生しました。"}), 500 # Internal Server Error

In [ ]:
# セル 7 内の start_ngrok_tunnel 関数を修正

def start_ngrok_tunnel(port):
    """指定されたポートでngrokトンネルを開始する"""
    global ngrok_tunnel
    logger.info("Attempting to start ngrok tunnel...")
    try:
        # 既存のトンネルがあれば切断 (コードは省略、前のまま)
        # ... (Disconnect existing tunnel logic) ...
        for tunnel in ngrok.get_tunnels():
             if tunnel.config['addr'].endswith(f':{port}'):
                 logger.warning(f"Disconnecting existing ngrok tunnel: {tunnel.public_url}")
                 ngrok.disconnect(tunnel.public_url)

        # 新しいトンネルを開始
        logger.info(f"Starting ngrok tunnel for port {port}...")
        ngrok_tunnel = ngrok.connect(port)

        # --- ★★★ デバッグ出力追加 ★★★ ---
        print(f"DEBUG [ngrok.connect completed]: Tunnel object = {ngrok_tunnel}") # connect直後のオブジェクト確認
        if ngrok_tunnel and hasattr(ngrok_tunnel, 'public_url') and ngrok_tunnel.public_url:
             print(f"DEBUG https://www.merriam-webster.com/dictionary/check: Public URL = {ngrok_tunnel.public_url}") # URLを強制表示
             logger.info(f"Ngrok tunnel started successfully! Public URL: {ngrok_tunnel.public_url}") # 元のログ
             return True
        else:
             # URLが取得できなかった場合
             logger.error("Ngrok tunnel object obtained, BUT failed to get a valid public_url.")
             print(f"DEBUG https://github.com/travis-ci/travis-ci/issues/9719: Tunnel object = {ngrok_tunnel}")
             return False
        # --- ★★★ デバッグ出力ここまで ★★★ ---

    except pyngrok_exception.PyngrokNgrokError as e:
         # ... (既存のエラー処理) ...
         logger.error(f"Failed to start ngrok tunnel (PyngrokNgrokError): {e}", exc_info=True)
         if "ERR_NGROK_4018" in str(e) or "authentication failed" in str(e).lower():
             logger.critical(">>> Ngrok authentication failed. Check your authtoken in Colab Secrets (Step 1).")
         elif "account limited" in str(e).lower():
             logger.error(">>> Your ngrok account might be limited (e.g., concurrent tunnels). Check your ngrok dashboard.")
         return False
    except Exception as e:
        # ... (既存のエラー処理) ...
        logger.error(f"An unexpected error occurred while starting ngrok: {e}", exc_info=True)
        return False

def shutdown_ngrok_tunnel():
    """ngrokトンネルを安全にシャットダウンする"""
    global ngrok_tunnel
    if ngrok_tunnel:
        logger.info(f"Disconnecting ngrok tunnel: {ngrok_tunnel.public_url}")
        try:
            ngrok.disconnect(ngrok_tunnel.public_url)
            ngrok_tunnel = None
            logger.info("Ngrok tunnel disconnected successfully.")
        except Exception as e:
            # 切断エラーは警告に留める
            logger.warning(f"Could not disconnect ngrok tunnel {ngrok_tunnel.public_url}: {e}")
    else:
        logger.info("No active ngrok tunnel to disconnect.")

def run_flask_app(port):
    """Flask開発サーバーを実行する"""
    logger.info(f"Starting Flask development server on port {port}...")
    # Werkzeug (Flaskの開発サーバー) のログレベルを WARNING に設定してアクセスログを抑制
    log = logging.getLogger('werkzeug')
    log.setLevel(logging.WARNING)
    # host='0.0.0.0' はコンテナ環境などで外部からアクセス可能にするためだが、
    # ngrokを使う場合は localhost (127.0.0.1) またはデフォルトで良い
    app.run(port=port) # debug=False が本番環境では推奨

# --- メイン実行ブロック ---
if __name__ == "__main__":
    logger.info("Starting main execution block...")
    # ngrokトンネルを開始
    if start_ngrok_tunnel(FLASK_PORT):
        try:
            # ngrok起動成功後、Flaskアプリを実行
            run_flask_app(FLASK_PORT)
        except KeyboardInterrupt:
            # Ctrl+C で中断した場合
            logger.info("Flask app interrupted by user (KeyboardInterrupt).")
        except Exception as e:
             # Flask実行中の予期せぬエラー
             logger.error(f"An error occurred while running the Flask app: {e}", exc_info=True)
        finally:
            # Flaskアプリ終了時にngrokをシャットダウン
            logger.info("Flask app finished or interrupted. Shutting down ngrok tunnel...")
            shutdown_ngrok_tunnel()
            logger.info("Application shutdown complete.")
    else:
        # ngrok起動失敗
        logger.error("Failed to start ngrok tunnel. Flask app will not run.")
        print("\n--- ngrokトンネルの起動に失敗しました ---")
        print("考えられる原因:")
        print("1. ngrok認証トークンがColabシークレットに正しく設定されていない (ステップ1を確認)。")
        print("2. ngrokアカウントの制限 (無料プランの同時接続数など)。")
        print("3. 一時的なネットワークの問題。")
        print("ログを確認し、問題を解決してから再実行してください。")

2025-05-03 04:55:39 - __main__ - INFO - Starting main execution block...
2025-05-03 04:55:39 - __main__ - INFO - Attempting to start ngrok tunnel...
2025-05-03 04:55:39 - pyngrok.process - INFO - Overriding default auth token
2025-05-03 04:55:39 - pyngrok.process.ngrok - INFO - t=2025-05-03T04:55:39+0000 lvl=info msg="no configuration paths supplied"
2025-05-03 04:55:39 - pyngrok.process.ngrok - INFO - t=2025-05-03T04:55:39+0000 lvl=info msg="using configuration at default config path" path=/root/.config/ngrok/ngrok.yml
2025-05-03 04:55:39 - pyngrok.process.ngrok - INFO - t=2025-05-03T04:55:39+0000 lvl=info msg="open config file" path=/root/.config/ngrok/ngrok.yml err=nil
2025-05-03 04:55:39 - pyngrok.process.ngrok - INFO - t=2025-05-03T04:55:39+0000 lvl=info msg="starting web service" obj=web addr=127.0.0.1:4040 allow_hosts=[]
2025-05-03 04:55:39 - pyngrok.process.ngrok - INFO - t=2025-05-03T04:55:39+0000 lvl=info msg="client session established" obj=tunnels.session
2025-05-03 04:55:3

DEBUG [ngrok.connect completed]: Tunnel object = NgrokTunnel: "https://205e-34-171-223-172.ngrok-free.app" -> "http://localhost:5000"
DEBUG https://www.merriam-webster.com/dictionary/check: Public URL = https://205e-34-171-223-172.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


2025-05-03 04:55:46 - pyngrok.process.ngrok - INFO - t=2025-05-03T04:55:46+0000 lvl=info msg="join connections" obj=join id=2a62c735e5d7 l=127.0.0.1:5000 r=220.96.68.141:57492
2025-05-03 04:55:47 - pyngrok.process.ngrok - INFO - t=2025-05-03T04:55:47+0000 lvl=info msg="join connections" obj=join id=a765077b97ff l=127.0.0.1:5000 r=220.96.68.141:57492
2025-05-03 04:56:00 - pyngrok.process.ngrok - INFO - t=2025-05-03T04:56:00+0000 lvl=info msg="join connections" obj=join id=e947c8eee8f6 l=127.0.0.1:5000 r=220.96.68.141:57492
2025-05-03 04:56:00 - __main__ - INFO - Received chat request with input: 'こんにちは'
2025-05-03 04:56:00 - __main__ - INFO - Generating model response...
2025-05-03 04:56:02 - __main__ - INFO - Model response generated.
2025-05-03 04:56:02 - __main__ - INFO - Processed response: 'こんにちは、編集部の助手です。助手は、編集部の仕事を手伝ってくれる、助手です。助手は、編集部の仕事を手伝ってくれる、助手です。助手'
2025-05-03 04:56:14 - pyngrok.process.ngrok - INFO - t=2025-05-03T04:56:14+0000 lvl=info msg="join connections" obj=join id=a9d

In [8]:
import shutil
import os

# MODEL_NAMEとMODEL_DIRの設定に合わせてパスを確認・修正してください
model_cache_name = "rinna_llama-3-youko-8b" # MODEL_NAME.replace('/', '_') と同じはず
cache_dir_to_delete = os.path.join("/content/drive/MyDrive/llama_model_cache", model_cache_name)

print(f"Attempting to delete cache directory: {cache_dir_to_delete}")
if os.path.exists(cache_dir_to_delete):
    try:
        shutil.rmtree(cache_dir_to_delete)
        print(f"Successfully deleted directory: {cache_dir_to_delete}")
    except Exception as e:
        print(f"Error deleting directory {cache_dir_to_delete}: {e}")
else:
    print(f"Cache directory not found: {cache_dir_to_delete}")

Attempting to delete cache directory: /content/drive/MyDrive/llama_model_cache/rinna_llama-3-youko-8b
Successfully deleted directory: /content/drive/MyDrive/llama_model_cache/rinna_llama-3-youko-8b
